In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [2]:
import pickle
import codecs
import collections
import keras.preprocessing
from sklearn.model_selection import KFold
import csv

Using TensorFlow backend.


In [3]:
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Reshape, Activation, Input, Dense,GRU,Reshape,TimeDistributed,Bidirectional,Dropout,Masking
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding,TimeDistributed,Flatten
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import GRU, Dropout
from keras.models import load_model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Masking
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
from keras import backend as K
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding,TimeDistributed,Flatten
from keras.layers import BatchNormalization,Dropout


In [4]:
import seaborn as sns
sns.set(style="darkgrid")
plt.rcParams['font.family']='TH SarabunPSK' #you can change to other font that works for you
plt.rcParams.update({'font.size': 50})

In [5]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [6]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
input_pbs = pd.read_csv('Thaipbs-notebook3.csv')

In [19]:
input_pbs.drop('Unnamed: 0', axis=1, inplace=True)

In [20]:
input_pbs.head()

,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar,year-month,month,year,sentiment
0,ฝุ่น PM2.5 : ทส.เตรียมตั้งศูนย์แก้ปัญหาหมอกควั...,05/04/2562,FRI,19:37,177,สิ่งแวดล้อม,"ฝุ่นเชียงใหม่,ฝุ่นPM2.5,ฝุ่นคลุมเมือง,เชียงใหม...",19,10,"ฝุ่น,PM,ทส.,เตรียม,ศูนย์,แก้,ปัญหา,หมอก,ควัน,ข...",11,43,2562-04,4,2562,pos
1,"เลือกตั้ง 2562 : ""เพื่อไทย"" เตรียมยื่น กกต.จัด...",05/04/2562,FRI,19:20,702,การเมือง,"เลือกตั้ง62,เพื่อ่ไทย,กกต.,นับคะแนน,ไทยพีบีเอส...",19,6,"เลือกตั้ง,2562,ไทย,เตรียม,ยื่น,กกต.,เลือกตั้ง,...",9,46,2562-04,4,2562,pos
2,เตรียมเอาผิด รพ.เอกชน 58 แห่งไม่ส่งข้อมูลราคายา,05/04/2562,FRI,19:18,583,สาธารณสุข,"ยา,เวชภัณฑ์ฯ,โรงพยาบาล,ค่าบริการทางการแพทย์,กร...",19,9,"เตรียม,รพ.เอกชน,ข้อมูล,ราคา,ยา",5,26,2562-04,4,2562,pos
3,ตั้งข้อสังเกตปลดล็อกกัญชา เอื้อนายทุนหรือไม่?,05/04/2562,FRI,19:16,928,อาชญากรรม,"กัญชา,มูลนิธิข้าวขวัญ,ThaiPBSnews",19,3,"ข้อ,สังเกต,ปลด,ล็อก,กัญชา,เอื้อ,ทุน",7,29,2562-04,4,2562,neg
4,"คำพิพากษาเต็ม ""เปรมชัย"" หลุดคดีครอบครองซากเสือดำ",05/04/2562,FRI,19:29,5163,สิ่งแวดล้อม,"เสือดำ,เปรมชัย,เขตรักษาพันธุ์สัตว์ป่าทุ่งใหญ่น...",19,5,"พิพากษา,เต็ม,เปรมชัย,หลุดคดี,ครอบครอง,ซาก,เสือดำ",7,42,2562-04,4,2562,neg


In [155]:
input_pbs['date'] = input_pbs['date'].apply(lambda x : x.split('/')[0]
                                            +'/'+x.split('/')[1]+'/'+str(int(x.split('/')[2])-543))
input_pbs['date'] = pd.to_datetime(input_pbs['date'])
input_pbs['timeDelta'] = pd.to_datetime('05/04/2019')- input_pbs['date']
input_pbs['timeDelta'] = input_pbs['timeDelta'].apply(lambda x: int(str(x).split(' ')[0]))

In [40]:
# input_pbs['headline'][0]

'ฝุ่น PM2.5 : ทส.เตรียมตั้งศูนย์แก้ปัญหาหมอกควันข้ามแดน'

In [90]:
# pos_tag('ฝุ่น,PM,ทส.,เตรียม,ศูนย์,แก้,ปัญหา,หมอก,ควัน,ข้าม,แดน'.split(','),engine='artagger')

[('ฝุ่น', 'NCMN'),
 ('PM', 'NCMN'),
 ('ทส.', 'NCMN'),
 ('เตรียม', 'VACT'),
 ('ศูนย์', 'NCMN'),
 ('แก้', 'VACT'),
 ('ปัญหา', 'NCMN'),
 ('หมอก', 'NCMN'),
 ('ควัน', 'NCMN'),
 ('ข้าม', 'VACT'),
 ('แดน', 'NCMN')]

In [70]:
from pythainlp.tag import pos_tag

In [141]:
input_pbs['num_noun'] = 0
input_pbs['num_pronoun'] = 0
input_pbs['num_verb'] = 0
input_pbs['num_preverb'] = 0
input_pbs['num_determiner'] = 0
input_pbs['num_adverb'] = 0
input_pbs['num_classifier'] = 0
input_pbs['num_conjunction'] = 0
input_pbs['num_preposition'] = 0
input_pbs['num_interjunction'] = 0
input_pbs['num_prefix'] = 0
input_pbs['num_ending'] = 0
input_pbs['num_negator'] = 0
input_pbs['num_punctuation'] = 0

In [101]:
# '“' == "''"

False

In [125]:
input_pbs['token'] = input_pbs['token'].apply(lambda x : x.replace('“,', ''))
input_pbs['token'] = input_pbs['token'].apply(lambda x : x.replace('”', ','))
input_pbs['token'] = input_pbs['token'].apply(lambda x : x.replace(',“', ','))
input_pbs['token'] = input_pbs['token'].apply(lambda x : x.replace(',,', ','))
input_pbs['token'] = input_pbs['token'].apply(lambda x : x.replace('“', ','))

In [126]:
# for index, row in input_pbs.iterrows():
#     if row['token'].find("“") != -1:
#         print(row['token'])

In [152]:
# pos_tag(['สวย'],engine='artagger')

[('สวย', 'VATT')]

In [142]:
for index, row in input_pbs.iterrows():
#     print(row['token'])
    pos = pos_tag(row['token'].split(','),engine='artagger')
#     print(pos)
    for token,p in pos:
        if p[0] =='NEG':
            input_pbs.at[index,'NEG'] += 1
        elif p[0] =='N':
            input_pbs.at[index,'num_noun'] += 1
#             print(ner.get_ner(token))
        elif p[0] =='PUNC':
            input_pbs.at[index,'num_punctuation'] += 1
        elif p[0] =='P':
            input_pbs.at[index,'num_pronoun'] += 1
        elif p[0] =='V':
            input_pbs.at[index,'num_verb'] += 1
        elif p[0] =='X':
            input_pbs.at[index,'num_preverb'] += 1
        elif p[0] =='D':
            input_pbs.at[index,'num_determiner'] += 1
        elif p[0] =='A':
            input_pbs.at[index,'num_adverb'] += 1
        elif p[0] =='C':
            input_pbs.at[index,'num_classifier'] += 1
        elif p[0] =='J':
            input_pbs.at[index,'num_conjunction'] += 1
        elif p[0] =='R':
            input_pbs.at[index,'num_preposition'] += 1
        elif p[0] =='I':
            input_pbs.at[index,'num_interjunction'] += 1
        elif p[0] =='F':
            input_pbs.at[index,'num_prefix'] += 1
        elif p[0] =='E':
            input_pbs.at[index,'num_ending'] += 1

In [173]:
input_pbs['nnoun_per_ntoken'] = input_pbs['num_noun']/input_pbs['numToken']
input_pbs['nverb_per_ntoken'] = input_pbs['num_verb']/input_pbs['numToken']
input_pbs['nadverb_per_ntoken'] = input_pbs['num_adverb']/input_pbs['numToken']

In [157]:
input_pbs.category.unique()

array(['สิ่งแวดล้อม', 'การเมือง', 'สาธารณสุข', 'อาชญากรรม', 'สังคม',
       'ต่างประเทศ', 'กีฬา', 'เศรษฐกิจ', 'ภูมิภาค',
       'วิทยาศาสตร์เทคโนโลยี', 'ศิลปะ-บันเทิง', 'ไลฟ์สไตล์', 'ภัยพิบัติ',
       'จังหวัดชายแดนภาคใต้', 'รายงานพิเศษ', 'บทความ-บทวิเคราะห์',
       'เกษตร', 'การศึกษา', 'สตรีและเยาวชน', 'อาเซียน'], dtype=object)

In [166]:
input_pbs['is_environment'] = (input_pbs['category']=='สิ่งแวดล้อม')
input_pbs['is_politic'] = (input_pbs['category']=='การเมือง')
input_pbs['is_health'] = (input_pbs['category']=='สาธารณสุข')
input_pbs['is_crime'] = (input_pbs['category']=='อาชญากรรม')
input_pbs['is_social'] = (input_pbs['category']=='สังคม')
input_pbs['is_inter'] = (input_pbs['category']=='ต่างประเทศ')
input_pbs['is_sport'] = (input_pbs['category']=='กีฬา')
input_pbs['is_economic'] = (input_pbs['category']=='เศรษฐกิจ')
input_pbs['is_region'] = (input_pbs['category']=='ภูมิภาค')
input_pbs['is_science'] = (input_pbs['category']=='วิทยาศาสตร์และเทคโนโลยี')
input_pbs['is_art'] = (input_pbs['category']=='ศิลปะ-บันเทิง')
input_pbs['is_lifestyle'] = (input_pbs['category']=='ไลฟ์สไตล์')
input_pbs['is_disaster'] = (input_pbs['category']=='ภัยพิบัติ')
input_pbs['is_south'] = (input_pbs['category']=='จังหวัดชายแดนภาคใต้')
input_pbs['is_special_report'] = (input_pbs['category']=='รายงานพิเศษ')
input_pbs['is_article'] = (input_pbs['category']=='บทความ-บทวิเคราะห์')
input_pbs['is_agriculture'] = (input_pbs['category']=='เกษตร')
input_pbs['is_education'] = (input_pbs['category']=='การศึกษา')
input_pbs['is_women'] = (input_pbs['category']=='สตรีและเยาวชน')
input_pbs['is_asian'] = (input_pbs['category']=='อาเซียน')

In [169]:
input_pbs['is_environment'] = input_pbs['is_environment'].astype(int)
input_pbs['is_politic'] = input_pbs['is_politic'].astype(int)
input_pbs['is_health'] = input_pbs['is_health'].astype(int)
input_pbs['is_crime'] = input_pbs['is_crime'].astype(int)
input_pbs['is_social'] = input_pbs['is_social'].astype(int)
input_pbs['is_inter'] = input_pbs['is_inter'].astype(int)
input_pbs['is_sport'] = input_pbs['is_sport'].astype(int)
input_pbs['is_economic'] = input_pbs['is_economic'].astype(int)
input_pbs['is_region'] = input_pbs['is_region'].astype(int)
input_pbs['is_science'] = input_pbs['is_science'].astype(int)
input_pbs['is_art'] = input_pbs['is_art'].astype(int)
input_pbs['is_lifestyle'] = input_pbs['is_lifestyle'].astype(int)
input_pbs['is_disaster'] = input_pbs['is_disaster'].astype(int)
input_pbs['is_south'] = input_pbs['is_south'].astype(int)
input_pbs['is_special_report'] = input_pbs['is_special_report'].astype(int)
input_pbs['is_article'] = input_pbs['is_article'].astype(int)
input_pbs['is_agriculture'] = input_pbs['is_agriculture'].astype(int)
input_pbs['is_education'] = input_pbs['is_education'].astype(int)
input_pbs['is_women'] = input_pbs['is_women'].astype(int)
input_pbs['is_asian'] = input_pbs['is_asian'].astype(int)

In [78]:
top_headline = np.asarray(pd.DataFrame(input_pbs.sort_values('view',ascending=False)[:1000]['headline'])['headline'])
top_headline_embed = np.asarray([sentence_vectorizer(x) for x in top_headline])
worst_headline = np.asarray(pd.DataFrame(input_pbs.sort_values('view',ascending=True)[:1000]['headline'])['headline'])
worst_headline_embed = np.asarray([sentence_vectorizer(x) for x in worst_headline])

In [79]:
worst_headline.shape

(1000,)

In [77]:
# top_headline[:10]

array(['กษัตริย์จิกมีแห่งภูฏาน-สมเด็จพระราชินีเจตซุน และพระโอรส เสด็จฯ ถึงประเทศไทย ',
       'พรุ่งนี้ เปิดจองเหรียญที่ระลึกพระราชพิธีถวายพระเพลิงพระบรมศพ',
       'เลือกตั้ง 2562 : ด่วน! กกต.เปิดผลนับคะแนน 75 %',
       'ฮือฮาอีก ! รับซื้อเหรียญ 5 หายาก ปี 2525 ราคา 4 แสนบาท',
       'ด่วน! สึกแล้ว "พุทธะอิสระ" คุมตัวไปเรือนจำกลางพิเศษกรุงเทพ',
       'แรงงานพันคนเคว้ง "โตโยต้า" เลิกจ้างกระทันหัน ยอมจ่ายชดเชย-โบนัสให้จบใน 1 สัปดาห์',
       'ฟังชัดๆ คลิปเสียงปริศนา ต่อรองหลังจับ "เปรมชัย" ',
       'รัฐบาล ยืนยัน ประกาศหยุดราชการ วันที่ 26 ต.ค. 60 เพียงวันเดียว ',
       'ตำรวจเผย "ครูจอมทรัพย์" ไม่ใช่ "แพะ" จ้างคนรับสารภาพแทน',
       'หลัง 29 พ.ย.นี้ครอบครอง “แหวนปลาโรนิน” ถูกปรับ 4 หมื่น'],
      dtype=object)

In [80]:
# worst_headline[:10]

array(['ตร.เตรียมรับมือการเดินทางกลับช่วงหยุดยาว-ตรวจจับผู้ทำผิดกฎหมายเคร่งครัด',
       'ลิเวอร์พูล เปิดบ้านชนะ 3-0 แมนเชสเตอร์ ซิตี้ ',
       '"สมยศ"ขอเอเอฟซีช่วยยกระดับผู้ตัดสินไทย',
       'เลือกตั้ง 2562 : ผู้สมัคร ส.ส.ปชป.ร้องสอบทุจริตเลือกตั้ง "พัทลุง"',
       'เปิดช่องทางจราจร 8 เลน เร่งระบายรถจากเหนือ-อีสานเข้ากรุง',
       '"วิษณุ" แจงจะใช้ ม.44 คุมประชามติกรณีเกิดเหตุวุ่นวาย',
       'ราชทัณฑ์ยันยันไม่มีร่องรอยทำร้าย "อาเดม" คาดสร้างสถานการณ์หวังผลทางคดี',
       'ทุกข์ของเกษตรอีสาน',
       'เตรียมช่อมแซมชั่วคราวถนนเลียบคลอง 13 ทรุดลึก 3 เมตร',
       'เมืองทอง ถล่ม โอสถสภา 8-3 คว้าแชมป์เลกแรก'], dtype=object)

In [81]:
def findMaxSim(embed1, headline_embed):
    a = np.asarray([cosine_similarity(embed1,x) for x in headline_embed])
    return a.max()

In [84]:
for index, row in input_pbs.iterrows():
    sent = row['headline']
    sent_embed = sentence_vectorizer(sent)
    input_pbs.at[index,'sim_top'] = findMaxSim(sent_embed,top_headline_embed)
    input_pbs.at[index,'sim_worst'] = findMaxSim(sent_embed,worst_headline_embed)

In [116]:
input_pbs['is_Mon'] = (input_pbs['DOW']=='MON')
input_pbs['is_Tue'] = (input_pbs['DOW']=='TUE')
input_pbs['is_Wed'] = (input_pbs['DOW']=='WED')
input_pbs['is_Thu'] = (input_pbs['DOW']=='THU')
input_pbs['is_Fri'] = (input_pbs['DOW']=='FRI')
input_pbs['is_Sat'] = (input_pbs['DOW']=='SAT')
input_pbs['is_Sun'] = (input_pbs['DOW']=='SUN')

In [117]:
input_pbs['is_Mon'] = input_pbs['is_Mon'].astype(int)
input_pbs['is_Tue'] = input_pbs['is_Tue'].astype(int)
input_pbs['is_Wed'] = input_pbs['is_Wed'].astype(int)
input_pbs['is_Thu'] = input_pbs['is_Thu'].astype(int)
input_pbs['is_Fri'] = input_pbs['is_Fri'].astype(int)
input_pbs['is_Sat'] = input_pbs['is_Sat'].astype(int)
input_pbs['is_Sun'] = input_pbs['is_Sun'].astype(int)
input_pbs['is_weekend'] = input_pbs['is_Sat']+input_pbs['is_Sun']

In [121]:
input_pbs.dropna(inplace=True)

In [132]:
tag_dict = dict()
for index, row in input_pbs.iterrows():
    tag = row['tag'].split(',')
    for t in tag:
        if t not in tag_dict.keys():
            tag_dict[t] = 1
        else:
            tag_dict[t] += 1

In [135]:
import operator
from collections import OrderedDict

In [182]:
# tag_dict

OrderedDict([('น้ำท่วม', 1209),
             ('เลือกตั้ง', 571),
             ('คสช.', 545),
             ('ระเบิด', 538),
             ('ตำรวจ', 515),
             ('เชียงราย', 514),
             ('พล.อ.ประยุทธ์ จันทร์โอชา', 509),
             ('ในหลวง', 494),
             ('นายกรัฐมนตรี', 492),
             ('เชียงใหม่', 462),
             ('พระบาทสมเด็จพระปรมินทรมหาภูมิพลอดุลยเดช', 419),
             ('กกต.', 416),
             ('ดีเอสไอ', 409),
             ('สหรัฐฯ', 406),
             ('จีน', 404),
             ('เลือกตั้ง 62', 391),
             ('เลือกตั้ง2562', 385),
             ('ถ้ำหลวง', 380),
             ('ฝนตก', 367),
             ('สงขลา', 357),
             ('เลือกตั้ง62', 338),
             ('เลือกตั้ง 2562', 333),
             ('สวรรคต', 327),
             ('ญี่ปุ่น', 320),
             ('อุบัติเหตุ', 317),
             ('วัดพระธรรมกาย', 306),
             ('กระบี่', 297),
             ('ยาเสพติด', 297),
             ('นครศรีธรรมราช', 296),
             ('ร่างรัฐธรร

In [207]:
len(tag_dict)

46645

In [191]:
input_pbs['tag_pop_sum'] = 0
input_pbs['tag_pop_mean'] = 0

In [192]:
for index, row in input_pbs.iterrows():
    tag = row['tag'].split(',')
    pop_list = [tag_dict[t] for t in tag if t in tag_dict.keys()]
    input_pbs.at[index,'tag_pop_sum'] = sum(pop_list)
    if len(pop_list) > 0:
        input_pbs.at[index,'tag_pop_mean'] = sum(pop_list)/len(pop_list)

In [275]:
top_news = pd.DataFrame(input_pbs.sort_values('view',ascending=False)[:1000])
worst_news = pd.DataFrame(input_pbs.sort_values('view',ascending=True)[:1000])

In [276]:
tag_dict_top = dict()
for index, row in top_news.iterrows():
    tag = row['tag'].split(',')
    for t in tag:
        if t not in tag_dict_top.keys():
            tag_dict_top[t] = 1
        else:
            tag_dict_top[t] += 1

In [277]:
tag_dict_worst = dict()
for index, row in worst_news.iterrows():
    tag = row['tag'].split(',')
    for t in tag:
        if t not in tag_dict_worst.keys():
            tag_dict_worst[t] = 1
        else:
            tag_dict_worst[t] += 1

In [278]:
del tag_dict_top['ไทยพีบีเอส']
del tag_dict_top['ThaiPBSnews']
del tag_dict_top['ข่าวไทยพีบีเอส']
del tag_dict_top['ThaiPBS']

In [279]:
del tag_dict_worst['ไทยพีบีเอส']
del tag_dict_worst['ThaiPBSnews']
del tag_dict_worst['ข่าวไทยพีบีเอส']
del tag_dict_worst['ThaiPBS']

In [280]:
len(tag_dict_top)

3466

In [281]:
len(tag_dict_worst)

2884

In [286]:
tag_intersect = set(tag_dict_top.keys()).intersection(set(tag_dict_worst.keys()))

In [287]:
len(tag_intersect)

492

In [291]:
tag_dict_top = {key: tag_dict_top[key] for key in tag_dict_top if key not in tag_intersect}
tag_dict_worst = {key: tag_dict_worst[key] for key in tag_dict_worst if key not in tag_intersect}

In [293]:
len(tag_dict_top)

2974

In [292]:
len(tag_dict_worst)

2392

In [294]:
input_pbs['tag_top_pop_sum'] = 0
input_pbs['tag_top_pop_mean'] = 0
input_pbs['num_tag_pop'] = 0
input_pbs['tag_top_worst_sum'] = 0
input_pbs['tag_top_worst_mean'] = 0
input_pbs['num_tag_worst'] = 0

In [295]:
for index, row in input_pbs.iterrows():
    tag = row['tag'].split(',')
    pop_list = [tag_dict_top[t] for t in tag if t in tag_dict_top.keys()]
    input_pbs.at[index,'num_tag_pop'] = len(pop_list)
    input_pbs.at[index,'tag_top_pop_sum'] = sum(pop_list)
    if len(pop_list) > 0:
        input_pbs.at[index,'tag_top_pop_mean'] = sum(pop_list)/len(pop_list)

In [296]:
for index, row in input_pbs.iterrows():
    tag = row['tag'].split(',')
    worst_list = [tag_dict_worst[t] for t in tag if t in tag_dict_worst.keys()]
    input_pbs.at[index,'num_tag_worst'] = len(worst_list)
    input_pbs.at[index,'tag_top_worst_sum'] = sum(worst_list)
    if len(worst_list) > 0:
        input_pbs.at[index,'tag_top_worst_mean'] = sum(worst_list)/len(worst_list)

In [297]:
len(set(tag_dict_top.keys()).intersection(set(tag_dict_worst.keys())))

0

In [12]:
input_pbs = pd.read_csv('notebook6-extractFeature-thispbs.csv')

In [13]:
input_pbs.drop('Unnamed: 0', axis=1, inplace=True)
# input_pbs.drop('Unnamed: 0.1', axis=1, inplace=True)

In [16]:
input_pbs[50:100]

,headline,date,DOW,time,view,category,tag,hour,numTag,token,numToken,numChar,year-month,month,year,sentiment,timeDelta,num_noun,num_pronoun,num_verb,num_preverb,num_determiner,num_adverb,num_classifier,num_conjunction,num_preposition,num_interjunction,num_prefix,num_ending,num_negator,num_punctuation,is_environment,is_politic,is_health,is_crime,is_social,is_inter,is_sport,is_economic,is_region,is_science,is_art,is_lifestyle,is_disaster,is_south,is_special_report,is_article,is_agriculture,is_education,is_women,is_asian,nnoun_per_ntoken,nverb_per_ntoken,nadverb_per_ntoken,num_date,num_email,num_law,num_len,num_location,num_money,num_organization,num_percent,num_person,num_phone,num_time,num_url,num_person_2,num_pos,num_neg,sim_top,sim_worst,is_Mon,is_Tue,is_Wed,is_Thu,is_Fri,is_Sat,is_Sun,is_weekend,tag_pop_sum,tag_pop_mean,tag_top_pop_sum,tag_top_pop_mean,num_tag_pop,tag_top_worst_sum,tag_top_worst_mean,num_tag_worst
50,สงกรานต์ 62 : สงกรานต์ คนกรุงฯ 11-14 เม.ย. เจอกันที่สวนลุมพินี,2019-04-04,THU,13:26,971,สังคม,"สงกรานต์,กรุงเทพฯ,กทม.,สวนลุมพินี,เทศกาลมหาสงกรานต์,ทำบุญตักบาตร,อัญเชิญพระพุทธสิหิงค์,รดน้ำขอพร,ประกวดเทพีสงกรานต์,ThaiPBSnews,ไทยพีบีเอส",13,11,"สงกรานต์,สงกรานต์,คนกรุง,เม.ย.,เจอ,สวนลุมพินี",6,40,2562-04,4,2562,neg,30,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.833333,0.166667,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0.756748,0.720363,0,0,0,1,0,0,0,0,588,65,0,0,0,1,1,1
51,4 เม.ย.เส้นตายให้ รพ.เอกชน ส่งข้อมูลราคายา,2019-04-04,THU,11:40,180,สาธารณสุข,"โรงพยาบาลเอกชน,ข้อมูลซื้อขายยา,ราคายา,ค่ารักษาพยาบาล,ไทยพีบีเอส,ข่าวไทยพีบีเอส",11,6,"เม.ย.,เส้นตาย,รพ.เอกชน,ข้อมูล,ราคา,ยา",6,32,2562-04,4,2562,pos,30,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.000000,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.634337,0.661619,0,0,0,1,0,0,0,0,29,7,0,0,0,0,0,0
52,"บุกค้น ""มูลนิธิข้าวขวัญสุพรรณ"" ยึดกัญชา 200 ต้น",2019-04-04,THU,11:15,14474,อาชญากรรม,"มูลนิธิข้าวขวัญ,กัญชา,สุพรรณบุรี,ไทยพีบีเอส,ข่าวไทยพีบีเอส",11,5,"บุกค้น,มูลนิธิข้าวขวัญ,สุพรรณ,ยึด,กัญชา,ต้น",6,38,2562-04,4,2562,neg,30,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.833333,0.166667,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.000000,0.528893,0,0,0,1,0,0,0,0,144,48,4,2,2,0,0,0
53,"เลือกตั้ง 2562 : ""ชัย ชิดชอบ"" ทำบุญวันเกิด 91 ปี นักการเมืองร่วมงานคับคั่ง",2019-04-04,THU,10:33,596,การเมือง,"เลือกตั้ง2562,ชัย,เนวิน,อนุทิน,ภูมิใจไทย",10,5,"เลือกตั้ง,2562,ชัยชิดชอบ,ทำ,บุญ,ปี,เมือง,งาน,คับคั่ง",9,44,2562-04,4,2562,pos,30,7,0,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.777778,0.222222,0.0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0.762091,0.732181,0,0,0,1,0,0,0,0,426,85,2,2,1,0,0,0
54,ฝุ่น PM2.5 : นักวิชาการคาด ภาคเหนือยังมีโอกาสเกิดฝุ่นละอองสะสม,2019-04-04,THU,07:06,469,ภูมิภาค,"ฝุ่น,เชียงใหม่,หมอก",7,3,"ฝุ่น,PM,วิชาการ,คาด,เหนือ,โอกาส,ฝุ่น,ละออง,สะสม",9,39,2562-04,4,2562,neg,30,6,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.666667,0.222222,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.641935,0.729164,0,0,0,1,0,0,0,0,603,201,0,0,0,0,0,0
55,"เลือกตั้ง 2562 : ถอดความหมาย ท่าที ""พล.อ.อภิรัชต์""",2019-04-04,THU,06:51,6518,การเมือง,"เลือกตั้ง2562,อภิรัชต์,ผบ.ทบ.,ประยุทธ์",6,4,"เลือกตั้ง,2562,ถอด,ความหมาย,ท่าที,พล.อ.อภิรัชต์",6,42,2562-04,4,2562,neg,30,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.833333,0.166667,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.722750,0.693686,0,0,0,1,0,0,0,0,540,135,0,0,0,6,6,1
56,"การใช้ถ้อยคำถวายพระพรชัยมงคล ""สมเด็จพระเจ้าอยู่หัว""",2019-04-04,THU,06:43,3513,สังคม,"พระราชพิธี,พระราชพิธีบรมราชาภิเษก,คำถวายพระพรชัยมงคลงานพระราชพิธี,สปน.",6,4,"ถ้อยคำ,ถวาย,พระพร,ชัยมงคล,สมเด็จพระเจ้าอยู่หัว",5,42,2562-04,4,2562,neg,30,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000,0.000000,0.0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0.609752,0.590255,0,0,0,1,0,0,0,0,33,8,3,1,2,0,0,0
57,ถมทรายฟื้นฟูหาดพัทยา หลังพายุฤดูร้อนถล่มรอบ 2,2019-04-04,THU,06:42,371,สิ่งแวดล้อม,"หาดพัทยา,พายุฤดูร้อน,พัทยา,ชายหาด,ThaiPBSnews,ไทยพีบีเอส,ข่าวไทยพีบีเอส",6,7,"ถมทรา

In [7]:
input_pbs.to_csv('notebook6-extractFeature-thispbs.csv')

NameError: name 'input_pbs' is not defined

In [300]:
input_pbs['view'].describe()

count    2.753800e+04
mean     2.455730e+03
std      1.272891e+04
min      2.000000e+01
25%      2.530000e+02
50%      5.620000e+02
75%      1.559000e+03
max      1.208350e+06
Name: view, dtype: float64

In [33]:
input_pbs.shape

(27974, 54)